# **Moive Recommendation**
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

# Data ETL and Data Exploration

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
movies_df = spark.read.load("drive/My Drive/ml-latest-small/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("drive/My Drive/ml-latest-small/ratings.csv", format='csv', header = True)
links_df = spark.read.load("drive/My Drive/ml-latest-small/links.csv", format='csv', header = True)
tags_df = spark.read.load("drive/My Drive/ml-latest-small/tags.csv", format='csv', header = True)

In [ ]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
ratings_df.show(5)

In [ ]:
links_df.show(5)

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows



In [ ]:
tags_df.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows



In [ ]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1


In [ ]:
tmp1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user


# Part 1: Spark SQL and OLAP

In [ ]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


### Q1: The number of Users

In [ ]:
q1_result=spark.sql("Select Count(Distinct userId) as Number_of_Users from ratings")
q1_result.show()

+---------------+
|Number_of_Users|
+---------------+
|            610|
+---------------+



### Q2: The number of Movies

In [ ]:
q2_result=spark.sql("Select Count(movieId) as Number_of_Moives from movies")
q2_result.show()

+----------------+
|Number_of_Moives|
+----------------+
|            9742|
+----------------+



### Q3:  How many movies are rated by users? List movies not rated before

In [ ]:
q3_result_1=spark.sql("Select Count(movieId) as Number_of_Rated_Moives From movies Where movieID in (Select movieId From ratings)")
q3_result_1.show()

+----------------------+
|Number_of_Rated_Moives|
+----------------------+
|                  9724|
+----------------------+



In [ ]:
q3_result_2=spark.sql("Select movieId, title From movies Where movieID not in (Select movieId From ratings)")
q3_result_2.show()

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|   1076|Innocents, The (1...|
|   2939|      Niagara (1953)|
|   3338|For All Mankind (...|
|   3456|Color of Paradise...|
|   4194|I Know Where I'm ...|
|   5721|  Chosen, The (1981)|
|   6668|Road Home, The (W...|
|   6849|      Scrooge (1970)|
|   7020|        Proof (1991)|
|   7792|Parallax View, Th...|
|   8765|This Gun for Hire...|
|  25855|Roaring Twenties,...|
|  26085|Mutiny on the Bou...|
|  30892|In the Realms of ...|
|  32160|Twentieth Century...|
|  32371|Call Northside 77...|
|  34482|Browning Version,...|
|  85565|  Chalet Girl (2011)|
+-------+--------------------+



### Q4: List Movie Genres

In [ ]:
q4_result=spark.sql("Select Distinct explode(split(genres,'[|]')) as genres From movies Order by 1")
q4_result.show()

+------------------+
|            genres|
+------------------+
|(no genres listed)|
|            Action|
|         Adventure|
|         Animation|
|          Children|
|            Comedy|
|             Crime|
|       Documentary|
|             Drama|
|           Fantasy|
|         Film-Noir|
|            Horror|
|              IMAX|
|           Musical|
|           Mystery|
|           Romance|
|            Sci-Fi|
|          Thriller|
|               War|
|           Western|
+------------------+



### Q5: Movie for Each Category

In [ ]:
q5_result_1=spark.sql("Select genres,Count(movieId) as Number_of_Moives From(Select explode(split(genres,'[|]')) as genres, movieId From movies) Group By 1 Order by 2 DESC")
q5_result_1.show()

+------------------+----------------+
|            genres|Number_of_Moives|
+------------------+----------------+
|             Drama|            4361|
|            Comedy|            3756|
|          Thriller|            1894|
|            Action|            1828|
|           Romance|            1596|
|         Adventure|            1263|
|             Crime|            1199|
|            Sci-Fi|             980|
|            Horror|             978|
|           Fantasy|             779|
|          Children|             664|
|         Animation|             611|
|           Mystery|             573|
|       Documentary|             440|
|               War|             382|
|           Musical|             334|
|           Western|             167|
|              IMAX|             158|
|         Film-Noir|              87|
|(no genres listed)|              34|
+------------------+----------------+



In [ ]:
q5_result_2=spark.sql("Select genres, concat_ws(',',collect_set(title)) as list_of_movies From(Select explode(split(genres,'[|]')) as genres, title From movies) Group By 1")
q5_result_2.show()

+------------------+--------------------+
|            genres|      list_of_movies|
+------------------+--------------------+
|             Crime|Stealing Rembrand...|
|           Romance|Vampire in Brookl...|
|          Thriller|Element of Crime,...|
|         Adventure|Ice Age: Collisio...|
|             Drama|Airport '77 (1977...|
|               War|General, The (192...|
|       Documentary|The Barkley Marat...|
|           Fantasy|Masters of the Un...|
|           Mystery|Before and After ...|
|           Musical|U2: Rattle and Hu...|
|         Animation|Ice Age: Collisio...|
|         Film-Noir|Rififi (Du rififi...|
|(no genres listed)|T2 3-D: Battle Ac...|
|              IMAX|Harry Potter and ...|
|            Horror|Underworld: Rise ...|
|           Western|Man Who Shot Libe...|
|            Comedy|Hysteria (2011),H...|
|          Children|Ice Age: Collisio...|
|            Action|Stealing Rembrand...|
|            Sci-Fi|Push (2009),SORI:...|
+------------------+--------------

# Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [ ]:
ratings_df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [ ]:
movie_ratings=ratings_df.drop('timestamp')

In [ ]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [ ]:
movie_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [ ]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [ ]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2])

In [ ]:
#Create ALS model
als = ALS(maxIter=5, rank=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [ ]:
#Tune model using ParamGridBuilder
paramGrid = (ParamGridBuilder()
             .addGrid(als.regParam, [0.05, 0.1, 0.3, 0.5])
             .addGrid(als.rank, [5, 10, 15])
             .addGrid(als.maxIter, [1, 5, 10])
             .build())

In [ ]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [ ]:
# Build Cross validation 
cv = CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [ ]:
#Fit ALS model to training data
cvModel = cv.fit(training)

In [ ]:
#Extract best model from the tuning exercise using ParamGridBuilder
bestModel=cvModel.bestModel

### Model testing
And finally, make a prediction and check the testing error.

In [ ]:
#Generate predictions and evaluate using RMSE
predictions=bestModel.transform(test)
rmse = evaluator.evaluate(predictions)

In [ ]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank: ", str(bestModel._java_obj.parent().getRank())),
print (" MaxIter: ", str(bestModel._java_obj.parent().getMaxIter())), 
print (" RegParam: ", str(bestModel._java_obj.parent().getRegParam()))

RMSE = 0.885871595187709
**Best Model**
 Rank:  5
 MaxIter:  10
 RegParam:  0.1


In [ ]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   322|   1580|   3.5| 3.1772356|
|   362|   1645|   5.0| 3.7667787|
|   368|   2122|   2.0| 2.4446008|
|   368|   2366|   4.0| 3.1004581|
|   385|    471|   4.0| 3.1240013|
|    28|   1580|   3.0| 2.8128936|
|   577|   1959|   4.0| 3.3411753|
|   271|   6658|   2.0| 3.0011191|
|   606|   1088|   3.0| 3.4075859|
|   606|   1580|   2.5| 3.1243372|
|   602|    471|   4.0| 2.9878833|
|   233|   1580|   3.0| 2.8116512|
|   599|   4519|   2.5| 2.6403553|
|   111|   4900|   4.0| 1.0384103|
|   325|   3918|   4.0| 3.2924266|
|   603|    471|   4.0| 3.0710413|
|   603|   3175|   4.0| 3.3628101|
|   274|   1645|   3.5| 3.1295524|
|   182|   1591|   3.5| 2.9039283|
|   280|   1580|   3.5| 3.3831246|
+------+-------+------+----------+
only showing top 20 rows



### Model apply and see the performance

In [ ]:
alldata=bestModel.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6930758075485765


In [ ]:
alldata.registerTempTable("alldata")

In [ ]:
spark.sql("Select * From alldata").show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   463|   1088|   3.5| 3.3877974|
|   137|   1580|   3.5| 3.3047454|
|   580|   1580|   4.0| 3.4698448|
|   580|   3175|   2.5| 3.3819396|
|   580|  44022|   3.5| 3.5123177|
|   133|    471|   4.0| 3.1618068|
|   322|   1580|   3.5|  3.201402|
|   362|   1591|   4.0|  3.415715|
|   362|   1645|   5.0|  3.723431|
|   593|   1580|   1.5|  2.692228|
|   597|    471|   2.0| 4.6544437|
|   597|   1580|   3.0| 3.5797641|
|   597|   1959|   4.0| 3.7440372|
|   597|   2366|   5.0| 4.1643567|
|   108|   1959|   5.0| 3.8643773|
|   155|   1580|   4.0| 3.6800473|
|   155|   3175|   4.0| 3.4354799|
|    34|   1580|   2.5| 3.1021104|
|    34|   3997|   2.0| 1.9403317|
|   368|   1580|   3.0| 2.9284894|
+------+-------+------+----------+
only showing top 20 rows



In [ ]:
spark.sql("select * from movies join alldata on movies.movieId=alldata.movieId").show()

+-------+--------------------+--------------------+------+-------+------+----------+
|movieId|               title|              genres|userId|movieId|rating|prediction|
+-------+--------------------+--------------------+------+-------+------+----------+
|   1088|Dirty Dancing (1987)|Drama|Musical|Rom...|   463|   1088|   3.5| 3.3877974|
|   1580|Men in Black (a.k...|Action|Comedy|Sci-Fi|   137|   1580|   3.5| 3.3047454|
|   1580|Men in Black (a.k...|Action|Comedy|Sci-Fi|   580|   1580|   4.0| 3.4698448|
|   3175| Galaxy Quest (1999)|Adventure|Comedy|...|   580|   3175|   2.5| 3.3819396|
|  44022|Ice Age 2: The Me...|Adventure|Animati...|   580|  44022|   3.5| 3.5123177|
|    471|Hudsucker Proxy, ...|              Comedy|   133|    471|   4.0| 3.1618068|
|   1580|Men in Black (a.k...|Action|Comedy|Sci-Fi|   322|   1580|   3.5|  3.201402|
|   1591|        Spawn (1997)|Action|Adventure|...|   362|   1591|   4.0|  3.415715|
|   1645|The Devil's Advoc...|Drama|Mystery|Thr...|   362|   1645

# Recommend moives to users with id: 575, 232. 
you can choose some users to recommend the moives 

In [ ]:
!pip install koalas
import databricks.koalas as ks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 390 kB 5.8 MB/s 


In [ ]:
userRecs = bestModel.recommendForAllUsers(10)

In [ ]:
userRecs_ks=userRecs.to_koalas()
movies_ks=movies_df.to_koalas()

In [ ]:
def movieRecommendation(inputId):
  recs_list=[]
  for recs in userRecs_ks.loc[str(inputId), 'recommendations']:
    recs_list.append(str(recs[0]))
  return (movies_ks[movies_ks['movieId'].isin(recs_list)])

In [ ]:
spark.sql("select * from movies join alldata on movies.movieId=alldata.movieId where userId = 575 order by rating desc limit 10").show()

+-------+--------------------+--------------------+------+-------+------+----------+
|movieId|               title|              genres|userId|movieId|rating|prediction|
+-------+--------------------+--------------------+------+-------+------+----------+
|    296| Pulp Fiction (1994)|Comedy|Crime|Dram...|   575|    296|   5.0| 4.7860193|
|    430|Calendar Girl (1993)|        Comedy|Drama|   575|    430|   5.0|  4.916182|
|   1259|  Stand by Me (1986)|     Adventure|Drama|   575|   1259|   5.0|    4.2128|
|   2542|Lock, Stock & Two...|Comedy|Crime|Thri...|   575|   2542|   5.0|  4.251685|
|   2622|William Shakespea...|      Comedy|Fantasy|   575|   2622|   5.0| 3.6177497|
|   2506|Other Sister, The...|Comedy|Drama|Romance|   575|   2506|   4.0| 3.8715992|
|   2567|         EDtv (1999)|              Comedy|   575|   2567|   4.0| 3.3972707|
|   2568|Mod Squad, The (1...|        Action|Crime|   575|   2568|   4.0| 1.3859453|
|   2571|  Matrix, The (1999)|Action|Sci-Fi|Thr...|   575|   2571

In [ ]:
print("Recommended movies for user with id '575' are as follows.")
movieRecommendation(575)

Recommended movies for user with id '575' are as follows.


,movieId,title,genres
27,28,Persuasion (1995),Drama|Romance
932,1232,Stalker (1979),Drama|Mystery|Sci-Fi
3749,5222,Kissing Jessica Stein (2001),Comedy|Romance
4590,6818,Come and See (Idi i smotri) (1985),Drama|War
5202,8477,"Jetée, La (1962)",Romance|Sci-Fi
6051,40491,"Match Factory Girl, The (Tulitikkutehtaan tytt...",Comedy|Drama
6697,58301,Funny Games U.S. (2007),Drama|Thriller
7277,74754,"Room, The (2003)",Comedy|Drama|Romance
7567,85774,Senna (2010),Documentary
9170,148881,World of Tomorrow (2015),Animation|Comedy


In [ ]:
spark.sql("select * from movies join alldata on movies.movieId=alldata.movieId where userId = 232 order by rating desc limit 10").show()

+-------+--------------------+--------------------+------+-------+------+----------+
|movieId|               title|              genres|userId|movieId|rating|prediction|
+-------+--------------------+--------------------+------+-------+------+----------+
|   5152|We Were Soldiers ...|    Action|Drama|War|   232|   5152|   5.0|  3.749648|
|   8533|Notebook, The (2004)|       Drama|Romance|   232|   8533|   5.0|  3.324488|
|   1210|Star Wars: Episod...|Action|Adventure|...|   232|   1210|   5.0|  3.923171|
|   3147|Green Mile, The (...|         Crime|Drama|   232|   3147|   5.0|  3.835532|
|   4226|      Memento (2000)|    Mystery|Thriller|   232|   4226|   5.0| 4.0841336|
|    296| Pulp Fiction (1994)|Comedy|Crime|Dram...|   232|    296|   5.0|   3.97758|
|  79132|    Inception (2010)|Action|Crime|Dram...|   232|  79132|   5.0|  4.102435|
|   2329|American History ...|         Crime|Drama|   232|   2329|   5.0| 4.0944633|
|  69757|(500) Days of Sum...|Comedy|Drama|Romance|   232|  69757

In [ ]:
print("Recommended movies for user with id '232' are as follows.")
movieRecommendation(232)

Recommended movies for user with id '232' are as follows.


,movieId,title,genres
3320,4495,Crossing Delancey (1988),Comedy|Romance
4251,6201,Lady Jane (1986),Drama|Romance
5025,7815,True Stories (1986),Comedy|Musical
5136,8235,Safety Last! (1923),Action|Comedy|Romance
5489,26326,"Holy Mountain, The (Montaña sagrada, La) (1973)",Drama
5867,32892,Ivan's Childhood (a.k.a. My Name is Ivan) (Iva...,Drama|War
5906,33649,Saving Face (2004),Comedy|Drama|Romance
7812,92494,Dylan Moran: Monster (2004),Comedy|Documentary
8154,102217,Bill Hicks: Revelations (1993),Comedy
9618,177593,"Three Billboards Outside Ebbing, Missouri (2017)",Crime|Drama


# Find the similar moives for moive with id: 463, 471

1.   列表项
2.   列表项


You can find the similar moives based on the ALS results

In [ ]:
itemFactors=bestModel.itemFactors.to_koalas()

In [ ]:
def similarMovies(inputId, matrix='cosine_similarity'):
  try:
    movieFeature=itemFactors.loc[itemFactors.id==str(inputId),'features'].to_numpy()[0]
  except:
    return 'There is no movie with id ' + str(inputId)
  
  if matrix=='cosine_similarity':
    similarMovie=pd.DataFrame(columns=('movieId','cosine_similarity'))
    for id,feature in itemFactors.to_numpy():
      cs=np.dot(movieFeature,feature)/(np.linalg.norm(movieFeature) * np.linalg.norm(feature))
      similarMovie=similarMovie.append({'movieId':str(id), 'cosine_similarity':cs}, ignore_index=True)
    similarMovie_cs=similarMovie.sort_values(by=['cosine_similarity'],ascending = False)[1:11]
    joint=similarMovie_cs.merge(movies_ks.to_pandas(), left_on='movieId', right_on = 'movieId', how = 'inner')
  if matrix=='euclidean_distance':
    similarMovie=pd.DataFrame(columns=('movieId','euclidean_distance'))
    for id,feature in itemFactors.to_numpy():
      ed=np.linalg.norm(np.array(movieFeature)-np.array(feature))
      similarMovie=similarMovie.append({'movieId':str(id), 'euclidean_distance':ed}, ignore_index=True)
    similarMovie_ed=similarMovie.sort_values(by=['euclidean_distance'])[1:11]
    joint=similarMovie_ed.merge(movies_ks.to_pandas(), left_on='movieId', right_on = 'movieId', how = 'inner')
  return joint[['movieId','title','genres']]

In [ ]:
print(similarMovies(463))

There is no movie with id 463


In [ ]:
spark.sql("select movies.movieId, title, genres from movies join alldata on movies.movieId=alldata.movieId where movies.movieId = 471 limit 1").show()

+-------+--------------------+------+
|movieId|               title|genres|
+-------+--------------------+------+
|    471|Hudsucker Proxy, ...|Comedy|
+-------+--------------------+------+



In [ ]:
print('Similar movies based on cosine similarity matrix are as follows.')
similarMovies(471, 'cosine_similarity')

Similar movies based on cosine similarity matrix are as follows.


,movieId,title,genres
0,5782,"Professional, The (Le professionnel) (1981)",Action|Drama|Thriller
1,3398,"Muppets Take Manhattan, The (1984)",Children|Comedy|Musical
2,151,Rob Roy (1995),Action|Drama|Romance|War
3,4047,Gettysburg (1993),Drama|War
4,25,Leaving Las Vegas (1995),Drama|Romance
5,48342,Conversations with Other Women (2005),Comedy|Drama|Romance
6,482,Killing Zoe (1994),Crime|Drama|Thriller
7,160567,Mike & Dave Need Wedding Dates (2016),Comedy
8,900,"American in Paris, An (1951)",Musical|Romance
9,37720,"Exorcism of Emily Rose, The (2005)",Crime|Drama|Horror|Thriller


In [ ]:
print('Similar movies based on euclidean distance matrix are as follows.')
similarMovies(471, 'euclidean_distance')

Similar movies based on euclidean distance matrix are as follows.


,movieId,title,genres
0,5782,"Professional, The (Le professionnel) (1981)",Action|Drama|Thriller
1,151,Rob Roy (1995),Action|Drama|Romance|War
2,4047,Gettysburg (1993),Drama|War
3,25,Leaving Las Vegas (1995),Drama|Romance
4,160567,Mike & Dave Need Wedding Dates (2016),Comedy
5,48342,Conversations with Other Women (2005),Comedy|Drama|Romance
6,3398,"Muppets Take Manhattan, The (1984)",Children|Comedy|Musical
7,1124,On Golden Pond (1981),Drama
8,62849,RocknRolla (2008),Action|Crime
9,482,Killing Zoe (1994),Crime|Drama|Thriller


# Write the report 
**motivation:** As artificial intelligence pervails in internet industry, more and more ecommerce platforms start to characterize their recommendation systems in order to provide better service. Collaborative filtering is one of the most popular recommendation algorithm which can be implemented with Alternating Least Squares (ALS) model in Spark ML. It would be a interesting and significant attempt to create a movie recommender for movie rating sites users.

**step1. Data ETL and Data Exploration**

I firstly loaded the rating data, established corresponding spark dataframes and checked out the basic information of the dataset.

**step2. Online Analytical Processing**

I performed analysis on the dataset from multi angle and gained some intuitive insights.

**step3. Model Selection**

I built up the ALS model and tuned the hyperparameter using 5-fold cross-validation, applying the optimal hyperparameters on the best final model.

**step4. Model Evaluation**

Finally, I evaluated the recommendation model by measuring the root-mean-square error of rating prediction on the testset. 

**step5. Model Application: Recommend moive to users**

For given users, I wrote a function to dirctly recommend 10 movies which they may be interested in based on the model.

**step6. Model Application: Find the similar moives**

I also applid the ALS results on finding the similar moives for a given movie. I used two matrix to evaluate the similarity between movies: cosine similarity and euclidean distance, which can be used sperately depends on situations.

**Output and Conclusion**

In this project, I built a ALS model with Spark APIs based on MovieLens dataset, predicted the ratings for the movies and made specific recommendation to users accordingly. The RMSE of the best model is approximately 0.88.